In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import yaml

In [41]:
exec(open('/home/msackel/Desktop/gammaClassification/programm/theta_cut/theta_cut.py').read()) 
exec(open('/home/msackel/Desktop/gammaClassification/programm/model_significance/model_significance.py').read()) 
 
with open('/home/msackel/Desktop/gammaClassification/config/feature.yaml') as f: 
        feature = yaml.load(f) 

In [71]:
train_data = theta_cut('/home/msackel/Desktop/gammaClassification/data/raw_data/gamma_precuts.hdf5',  
                '/home/msackel/Desktop/gammaClassification/data/raw_data/mrk501_2014_precuts.hdf5', 0.1,100000)  

In [84]:
xgbc = XGBClassifier(
max_depth= 3,
    booster= 'gbtree',
    n_jobs= 25,
    eval_metric= 'auc',
    gamma= 0.1,
    min_child_weight= 0.01,
    subsample=0.2,
    learning_rate= 0.2
)

parametergrid = {
    'reg_lambda':[3,4,2]
}

In [85]:
clf = GridSearchCV(
        xgbc,
        param_grid= parametergrid,
        scoring='roc_auc',
        cv=3
        )

In [86]:
clf.fit(train_data.drop('label', axis=1), train_data.label)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0.1, learning_rate=0.2,
       max_delta_step=0, max_depth=3, min_child_weight=0.01, missing=None,
       n_estimators=100, n_jobs=25, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.2),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'reg_lambda': [3, 4, 2]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='roc_auc', verbose=0)

In [87]:
'''
Save the scorrings of the different parameters.
'''
best_esti = pd.DataFrame(clf.cv_results_)
best_esti.to_pickle('bestEsti3')

In [88]:
best_esti = pd.read_pickle('bestEsti3')
a = np.argsort(best_esti['mean_test_score'].values)
best_esti.loc[a[-1]]['params']

{'reg_lambda': 3}

In [83]:
best_esti.loc[a[::-1]]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_reg_lambda,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,1.120167,0.035973,0.852320,0.856066,3,{'reg_lambda': 3},1,0.849944,0.857401,0.850150,0.856984,0.856867,0.853814,0.064974,0.000622,0.003216,0.001602
1,1.091224,0.037492,0.852121,0.855774,5,{'reg_lambda': 5},2,0.849426,0.856874,0.849634,0.856637,0.857301,0.853811,0.015331,0.000807,0.003664,0.001391
2,1.110102,0.036241,0.851373,0.854476,30,{'reg_lambda': 30},3,0.849162,0.855981,0.849098,0.855109,0.855860,0.852337,0.006855,0.000356,0.003173,0.001554
3,1.187494,0.041275,0.850155,0.852573,100,{'reg_lambda': 100},4,0.847951,0.854049,0.847632,0.853405,0.854881,0.850266,0.049520,0.004747,0.003344,0.001653


In [25]:
xgbc = XGBClassifier(
    max_depth= 3,
    booster= 'gbtree',
    n_jobs= 15,
    subsample= 0.25,
    eval_metric= 'auc',
    gamma = 0.1,
    learning_rate= 1,
    max_delta_step = 10,
    n_estimators = 50,
    reg_lambda = 0.001
)

In [62]:
cross_val_score(xgbc, data.drop('label', axis=1), data.label).mean()

NameError: name 'data' is not defined